In [2]:
with open (path, 'wb') as file:
    target = pickle.load(file)

NameError: name 'path' is not defined

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_estimator as tfe

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
master_test = pd.read_csv('./data/master_test.csv')

In [3]:
master_test=master_test.drop(['year','YEAR'], axis=1)

In [4]:
master_clean=master_test[(master_test['Year'] > 2000) & (master_test['Year'] <2019)]

In [5]:
master_clean

,Year,USCPI,GDPUSD,IYR,EYR,USDBarrel,USintRate,EUCPI,EUGDP,EXRATEUSEU
1,2018,251.104167,20891.367,487915.6285,318619.3433,71.335000,1.843012,103.891667,2920293.4,1.1456
2,2017,245.134167,19831.829,434632.6375,283269.2569,54.124805,0.851000,101.958333,2844787.0,1.2022
3,2016,240.007750,18979.245,416393.3358,269548.7079,43.638000,0.254520,100.250000,2734266.9,1.0552
4,2015,236.997833,18354.372,427810.2682,271911.1880,52.316549,0.036016,100.000000,2662582.6,1.0859
5,2014,236.715000,17838.454,420608.8378,276274.4108,98.969606,0.028200,99.891667,2571757.1,1.2101
6,2013,232.951750,17083.137,387510.3699,262095.3331,108.555000,0.046280,99.358333,2506973.4,1.3779
7,2012,229.586083,16358.863,381755.3631,265372.5075,111.570683,0.070760,97.866667,2462334.2,1.3186
8,2011,224.923000,15796.460,368463.9312,269068.6840,111.264274,0.039600,95.358333,2457515.0,1.2973
9,2010,218.076167,15240.843,319263.8226,239590.9601,79.609444,0.114542,92.491667,2417563.2,1.3269
10,2009,214.564667,14628.021,281800.8467,220599.3458,61.738770,0.096360,90.616667,2340629.1,1.4332


In [6]:
master_sample = master_clean

In [63]:
y_val = master_sample.pop('EXRATEUSEU')

dump= master_sample.pop('Year')

In [64]:
x_data = master_sample

In [65]:
X_train, X_test,y_train,y_test=train_test_split(x_data,y_val,test_size=0.3,random_state=101)

In [66]:
scaler_model = MinMaxScaler()

scaler_model.fit(X_train)

X_train=pd.DataFrame(scaler_model.transform(X_train),columns=X_train.columns,index=X_train.index)

scaler_model.fit(X_test)
X_test=pd.DataFrame(scaler_model.transform(X_test),columns=X_test.columns,index=X_test.index)

In [67]:
#Creating Feature Columns

def get_feat_cols_tf_format():
    feat_cols=[]
    for cols in master_sample.columns:
        column=tf.feature_column.numeric_column(cols)
        feat_cols.append(column)
    return feat_cols


In [68]:
tf_cols=get_feat_cols_tf_format()
tf_cols

[NumericColumn(key='USCPI', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='GDPUSD', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='IYR', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EYR', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='USDBarrel', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='USintRate', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EUCPI', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EUGDP', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [69]:
# Construct estimator model
# model=tf.estimator.DNNLinearCombinedRegressor(linear_feature_columns=tf_cols )
# tf.config.rnn.LSTMCell
model=tf.estimator.DNNRegressor(hidden_units=[6,10,6],feature_columns=tf_cols, activation_fn='LTSM')


W0328 12:44:19.907323 16736 estimator.py:1799] Using temporary folder as model directory: C:\Users\ORSONK~1\AppData\Local\Temp\tmpwaj3hsfs


In [70]:
type(X_train)

pandas.core.frame.DataFrame

In [71]:
input_func=tfe.estimator.inputs.pandas_input_fn(X_train,y_train,batch_size=3,num_epochs=3,shuffle=True)

In [72]:
input_func

<function tensorflow_estimator.python.estimator.inputs.pandas_io.pandas_input_fn.<locals>.input_fn()>

In [73]:
model.train(input_fn=input_func,steps=3)

ValueError: Unknown activation function:LTSM

In [24]:
train_metrics=model.evaluate(input_fn=input_func,steps=3)

W0328 10:22:22.781881 16736 deprecation.py:323] From C:\Users\OrsonKong\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [25]:
pred_input_func=tfe.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=3,num_epochs=1,shuffle=False)

In [26]:
preds=model.predict(input_fn=pred_input_func)

In [27]:
pred_input_func

<function tensorflow_estimator.python.estimator.inputs.pandas_io.pandas_input_fn.<locals>.input_fn()>

In [28]:
predictions=list(preds)

In [29]:
final_pred=[]

for pred in predictions:

    final_pred.append(pred["predictions"])

In [30]:
final_pred

[array([0.07554869], dtype=float32),
 array([0.03103648], dtype=float32),
 array([0.08709181], dtype=float32),
 array([0.07574892], dtype=float32),
 array([0.10527188], dtype=float32),
 array([0.0182212], dtype=float32)]

In [198]:
for e in final_pred:    
    print(e)

[0.47126916]
[0.5134931]
[0.5158901]
[0.5023085]
[0.52561474]
[0.7446378]
